In [ ]:
from langsmith import Client
from langsmith.utils import LangSmithNotFoundError

# Your questions and answers
# Define questions and answers for the golden dataset
question_answers = {
    # Basic Retrieval
    "What is the amount of men in Prague at the end of Q3 2024?": "676069",
    "What is the amount of women in Prague at the end of Q3 2024?": "716056",
    "What is the amount of women in Zlin region at the end of Q3 2024?": "294996",
    "What was the total population of Czech Republic at the start of Q1-Q3 2024?": "10900555",
    "Show me the female population in South Moravia at mid-2024?": "625214",
    "What was the middle-period population count for Zlín Region?": "579562",

    # Comparisons
    "Did Prague have more residents than Central Bohemia at the start of 2024?":
        "No (1384732 < 1455940)",
    "Which region had higher female population - Liberec or Karlovy Vary at period end?":
        "Liberec (228648 > 150270)",
    "List regions where mid-period male population exceeded 300,000":
        "Central Bohemia, Prague, South Moravia, Moravian-Silesia, Pilsner, South Bohemia, Olomouc, Ústí",
    "Was there any quarter where Plzeň's women outnumbered men by >15,000?":
        "No",

    # Aggregations
    "What was the average male population across all regions at period end?": "381994.5",
    "Which region had the lowest total population at the end of Q3?": "Karlovy Vary (293218)",
    "Count how many regions had <500k total residents mid-period": "2",
    "What was the maximum female population recorded in any region?": "743608",

    # Temporal/Deltas
    "How much did Prague's population grow from start to end of Q3?": "+7393",
    "Calculate percentage change in Ústí's male population (start vs end)": "-0.37%",
    "Which region had the steepest population decline?": "Moravian-Silesia (-5730)",
    "What was South Bohemia's population change rate per month?": "-128.77 per month",

    # Demographic Ratios
    "What percentage of Pardubice region was female at mid-period?": "50.57%",
    "What was the male:female ratio in Vysočina at period start?": "0.985",
    "Which region had the most balanced gender ratio at period end?": "Vysočina",
    "How many women per 100 men in Olomouc at mid-year?": "104.4",

    # Complex Joins/Logic
    "Compare Prague's start vs end population for men and women":
        "Men: +5389, Women: +2004",
    "Rank regions by population growth rate from start to end":
        "1. Prague (+0.53%), 2. Central Bohemia (+0.46%), 3. South Moravia (+0.06%), 4. Czech Republic (-0.03%), 5. Pilsner (-0.04%)",
    "List regions where women population decreased >1000":
        "Ústí (-1466), Moravian-Silesia (-3299)",
    "Show regions where mid-period population was < start but > end":
        "there are no such cases",

    # Edge Cases
    "What if Karlovy Vary had no data for mid-period women count?":
        "Not true, those data are available",
    "Show regions with exactly 500,000 residents at any point":
        "there are None",
    "Find records where male count equals female count":
        "there are None",
    "List regions with unreported start-period data":
        "there are None (all reported)"
}

ls_client = Client()

# Get or create dataset
try:
    dataset = ls_client.read_dataset(dataset_name="czsu agent")
    print(f"Dataset '{dataset.name}' found with ID: {dataset.id}")
except LangSmithNotFoundError:
    dataset = ls_client.create_dataset(
        dataset_name="czsu agent",
        description="Dataset of Czech Statistical Office agent questions and answers."
    )
    print(f"Dataset '{dataset.name}' created with ID: {dataset.id}")

# Fetch existing examples' questions to avoid duplicates
existing_examples = ls_client.list_examples(dataset_id=dataset.id)
existing_questions = set()
for ex in existing_examples:
    q = ex.inputs.get("question")
    if q:
        existing_questions.add(q)

# Prepare only new examples (filter out duplicates)
new_examples = []
for question, answer in question_answers.items():
    if question not in existing_questions:
        new_examples.append({"inputs": {"question": question}, "outputs": {"answers": answer}})

if new_examples:
    ls_client.create_examples(dataset_id=dataset.id, examples=new_examples)
    print(f"Added {len(new_examples)} new examples to dataset '{dataset.name}'.")
else:
    print("No new examples to add; all questions already exist in the dataset.")

Dataset 'czsu agent' found with ID: e585a306-7113-491d-ba41-bc03c09eb612
Added 30 new examples to dataset 'czsu agent'.
